# DSC Donation Details

### Import Libraries

In [1]:
import sys
import pandas
import numpy
from pandas               import DataFrame
from pandas               import read_csv
from numpy                import where
from numpy                import array
from numpy                import random
from sklearn.linear_model import LogisticRegression 
from sklearn.metrics      import auc
from sklearn.metrics      import roc_auc_score
from matplotlib           import pyplot
from scipy.stats          import pearsonr

### Local path

In [2]:
sys.path.append("C:/Users/abannem/Documents/MSc In Big Data For Business/Courses/12. Descriptive & Predictive Analytics/Group Project")

In [3]:
from ieseg import partition
from ieseg import roc

### Path

In [7]:
directory = "C:/Users/abannem/Documents/MSc In Big Data For Business/Courses/12. Descriptive & Predictive Analytics/Group Project"
inputData = directory + "/data"
sandbox   = directory + "/Sandbox"

### Import Data

In [8]:
# Train dataset
campaign2013 = read_csv(inputData + "/campaign20130411.csv", sep = ';')

FileNotFoundError: [Errno 2] File b'C:/Users/abannem/Documents/MSc In Big Data For Business/Courses/12. Descriptive & Predictive Analytics/Group Project/data/campaign20130411.csv' does not exist: b'C:/Users/abannem/Documents/MSc In Big Data For Business/Courses/12. Descriptive & Predictive Analytics/Group Project/data/campaign20130411.csv'

In [6]:
#Grouping the data by DonorID to find the total amount donated
campaign2013 = campaign2013.groupby(['donorID'],as_index=False).agg('sum')

NameError: name 'campaign2013' is not defined

In [330]:
#Rename Amount Column
campaign2013 = campaign2013.rename(columns= {'amount': 'amount_campaign'})

In [331]:
# Donor dataset
donors = read_csv(inputData + "/donors.csv", sep = ';', na_values = 'Missing') #Assigning missing values

In [332]:
#columns Added for Gender
#Coumn 1,2,3,4,5
donors['is_donor_M'] = where(donors["gender"] =='M', 1,0)
donors['is_donor_F'] = where(donors["gender"] =='F', 1,0)
donors['is_donor_C'] = where(donors["gender"] =='C', 1,0)
donors['is_donor_S'] = where(donors["gender"] =='S', 1,0)
donors['is_donor_U'] = where(donors["gender"] =='U', 1,0)

In [333]:
#column Added for Language
#Coumn 6,7
donors['is_donor_French'] = where(donors["language"] =='F', 1,0)


In [334]:
#Handle Null Colums in Zipcode
donors['zipcode'] = where(donors["zipcode"].isnull(), 0,donors["zipcode"])

In [335]:
#Handle one string value in Zipcode(No such zipcode in Brussel)
donors['zipcode'] = where(donors["zipcode"] =='SW6 3PS',"10000",donors['zipcode'])

In [336]:
#Change Zipcode type to Numeric
donors['zipcode']= pandas.to_numeric(donors['zipcode'])

In [337]:
donors['province'] = "London"

In [338]:
#Assign real values to province using Zipcode
donors['province'] = where((donors['zipcode'] >= 1000) & (donors['zipcode']<=1299), "Brussels capital Region",donors['province'])
donors['province'] = where((donors['zipcode'] >= 1300) & (donors['zipcode']<=1499), "Walloon Brabant",donors['province'])
donors['province'] = where((donors['zipcode'] >= 1500) & (donors['zipcode']<=1999), "Flemish Brabant",donors['province'])
donors['province'] = where((donors['zipcode'] >= 2000) & (donors['zipcode']<=2999), "Antwerp",donors['province'])
donors['province'] = where((donors['zipcode'] >= 3000) & (donors['zipcode']<=3499), "Flemish Brabant2",donors['province'])
donors['province'] = where((donors['zipcode'] >= 3500) & (donors['zipcode']<=3999), "Limburg",donors['province'])
donors['province'] = where((donors['zipcode'] >= 4000) & (donors['zipcode']<=4999), "Liège",donors['province'])
donors['province'] = where((donors['zipcode'] >= 5000) & (donors['zipcode']<=5999), "Namur",donors['province'])
donors['province'] = where((donors['zipcode'] >= 6000) & (donors['zipcode']<=6599), "Hainaut",donors['province'])
donors['province'] = where((donors['zipcode'] >= 6600) & (donors['zipcode']<=6999), "Luxembourg",donors['province'])
donors['province'] = where((donors['zipcode'] >= 7000) & (donors['zipcode']<=7999), "Hainaut2",donors['province'])
donors['province'] = where((donors['zipcode'] >= 8000) & (donors['zipcode']<=8999), "West Flanders",donors['province'])
donors['province'] = where((donors['zipcode'] >= 9000) & (donors['zipcode']<=9999), "East Flanders",donors['province'])

In [339]:
#Column Conversion for Province
donors['Is_Brussel'] = where(donors['province'] == "Brussels capital Region", 1,0)
donors['Is_Walloon'] = where(donors['province'] == "Walloon Brabant", 1,0)
donors['Is_FlemishBrabant'] = where(donors['province'] == "Flemish Brabant", 1,0)
donors['Is_Antwerp'] = where(donors['province'] == "Antwerp", 1,0)
donors['Is_FlemishBrabant2'] = where(donors['province'] == "Flemish Brabant2", 1,0)
donors['Is_Limburg'] = where(donors['province'] == "Limburg", 1,0)
donors['Is_Liège'] = where(donors['province'] == "Liège", 1,0)
donors['Is_Namur'] = where(donors['province'] == "Namur", 1,0)
donors['Is_Hainaut'] = where(donors['province'] == "Hainaut", 1,0)
donors['Is_Luxembourg'] = where(donors['province'] == "Luxembourg", 1,0)
donors['Is_Hainaut2'] = where(donors['province'] == "Hainaut2", 1,0)
donors['Is_WestFlanders'] = where(donors['province'] == "West Flanders", 1,0)
donors['Is_EastFlanders'] = where(donors['province'] == "East Flanders", 1,0)


In [340]:
#Assign real values to region based on province
donors['region'] = where(donors['province'] == "Brussels capital Region", "Brussels",donors['region'])
donors['region'] = where(donors['province'] == "Walloon Brabant", "Wallonia",donors['region'])
donors['region'] = where(donors['province'] == "Liège", "Wallonia",donors['region'])
donors['region'] = where(donors['province'] == "Namur", "Wallonia",donors['region'])
donors['region'] = where(donors['province'] == "Luxembourg", "Wallonia",donors['region'])
donors['region'] = where(donors['province'] == "Hainaut", "Wallonia",donors['region'])
donors['region'] = where(donors['province'] == "Hainaut2", "Wallonia",donors['region'])
donors['region'] = where(donors['province'] == "Flemish Brabant", "Flanders",donors['region'])
donors['region'] = where(donors['province'] == "Antwerp", "Flanders",donors['region'])
donors['region'] = where(donors['province'] == "Flemish Brabant2", "Flanders",donors['region'])
donors['region'] = where(donors['province'] == "Limburg", "Flanders",donors['region'])
donors['region'] = where(donors['province'] == "West Flanders", "Flanders",donors['region'])
donors['region'] = where(donors['province'] == "East Flanders", "Flanders",donors['region'])

In [341]:
#Column Conversion for Region
donors['Is_Brussel_region'] = where(donors['region'] == "Brussels", 1,0)
donors['Is_Wallonia_region'] = where(donors['region'] == "Wallonia", 1,0)


In [342]:
# gifts dataset
gifts = read_csv(inputData + "/gifts.csv", sep = ';')

In [343]:
#renaming the amount column in Gifts
gifts = gifts.rename(columns= {'amount': 'amount_gifts'})

In [344]:
#Convert to date format
gifts['date'] = pandas.to_datetime(gifts['date'])

In [345]:
#filtering data before the campaign date
gifts = gifts[gifts.date <= '2013-04-11']

In [346]:
#Camp count per year
camp_count = gifts.groupby(['donorID'],as_index=False).count()[['campID']]

In [347]:
#column 8
camp_count['is_camp_attended'] = where(camp_count["campID"] > 0, 1,0)

In [348]:
#column 9
#Total Donation per donor
donation_donor = gifts.groupby(['donorID'],as_index=False).sum()[['amount_gifts']]

In [349]:
#create a new column for year to obtain the donations per year
gifts['gifts_year'] = gifts['date'].dt.year

In [350]:
# - to check if it is required qs it has more than one row for donor id
#sum the donations to obtain per year
donation_year = gifts.groupby(['donorID','gifts_year'],as_index=False).sum()[['amount_gifts']]

In [351]:
donation_year_freq = gifts.groupby(['donorID'],as_index=False).sum()[['amount_gifts']]

In [352]:
#filtering data for the 3 year frequency rate
freq_3 = gifts[(gifts['date'] >= '2010-04-11') & (gifts['date'] <= '2013-04-11')]
#column 11
#Camp count freq per 3 years
freq_3_camp = freq_3.groupby(['donorID'],as_index=False).count()[['campID']]
freq_3_camp['is_freq_3_camp_attended'] = where(freq_3_camp["campID"] >0, 1,0)
#column 12
#Donation freq per 3 years
Donation_last_3_years = freq_3.groupby(['donorID'],as_index=False).sum()[['amount_gifts']]


In [353]:
#filtering data for the 5 year frequency rate
freq_5 = gifts[(gifts['date'] >= '2008-04-11') & (gifts['date'] <= '2013-04-11')]
#column 13
##Camp count freq per 5 years
freq_5_camp = freq_5.groupby(['donorID'],as_index=False).count()[['campID']]
freq_5_camp['is_freq_5_camp_attended'] = where(freq_5_camp["campID"] >0, 1,0)
#column 14
#Donation freq per 5 years
Donation_last_5_years = freq_5.groupby(['donorID'],as_index=False).sum()[['amount_gifts']]

In [354]:
#column 15
#max donation by a donor
max_donation = gifts.groupby(['donorID','gifts_year'],as_index=False).sum()
max_donation = max_donation.groupby(['donorID'],as_index=False).max()[['donorID','amount_gifts']]

In [355]:
#column 16
#min donation by a donor
min_donation = gifts.groupby(['donorID','gifts_year'],as_index=False).sum()
min_donation = min_donation.groupby(['donorID'],as_index=False).min()[['donorID','amount_gifts']]

In [356]:
#column 17
#median donation by a donor
#median_donation = gifts.groupby(['donorID','gifts_year'],as_index=False).sum()
#median_donation = median_donation.groupby(['donorID'],as_index=False).median()[['donorID','amount_gifts']]

In [357]:
#column 18
#Max Amount Paid by a donor
Max_Amount_Donated=gifts.groupby(['donorID'],as_index=False).max()[['donorID','amount_gifts']]
#column 19
#Min Amount Paid by a donor
Min_Amount_Donated=gifts.groupby(['donorID'],as_index=False).min()[['donorID','amount_gifts']]
#column 20
#Median Amount Paid by a donor
#Median_Amount_Donated=gifts.groupby(['donorID'],as_index=False).median()[['donorID','amount_gifts']]


In [358]:
#column 21
#Fist Amount Donated
First_Amount_Donated=gifts.groupby(['donorID','date'],as_index=False).min()[['donorID','amount_gifts','date']]
#column 22
#Last Amount Donated
Last_Amount_Donated=gifts.groupby(['donorID','date'],as_index=False).max()[['donorID','amount_gifts','date']]

In [359]:
#column 23
#Time since first donation
gb = gifts.groupby(['donorID'],as_index=False)
gb=gb.agg({'date' : numpy.min})
gb_update= gb['date'].dt.year
Years_from_first_donation =gb_update.to_frame()
Years_from_first_donation['Years_from_first_donation'] = 2013-Years_from_first_donation['date']

In [360]:
#column 24
#Time since last donation
gb = gifts.groupby(['donorID'],as_index=False)
gb=gb.agg({'date' : numpy.max})
gb_update= gb['date'].dt.year
Years_from_last_donation =gb_update.to_frame()
Years_from_last_donation['Years_from_last_donation'] = 2013-Years_from_last_donation['date']

In [361]:
train = campaign2013

In [362]:
train['is_camp_attended'] = camp_count['is_camp_attended']

In [363]:
train['total_amount_gifts'] = donation_donor['amount_gifts']

In [364]:
train['is_donation_year_GE_35'] = where(train['total_amount_gifts'] > 35, 1,0)

In [365]:
train['is_donation_year_GE_60'] = where(train['total_amount_gifts'] > 60, 1,0)

In [366]:
train['is_donation_year_GE_100'] = where(train['total_amount_gifts'] > 100, 1,0)

In [367]:
train['is_freq_3_camp_attended']=freq_3_camp['is_freq_3_camp_attended']

In [368]:
train['is_freq_5_camp_attended']=freq_5_camp['is_freq_5_camp_attended'] 

In [369]:
train['Donation_last_3_years']=Donation_last_3_years['amount_gifts']

In [370]:
train['Donation_last_5_years']=Donation_last_5_years['amount_gifts']

In [371]:
train['max_amount_donated']=max_donation['amount_gifts']

In [372]:
train['min_amount_donated']=min_donation['amount_gifts']

In [373]:
train['median_amount_donated']=median_donation['amount_gifts']

In [374]:
train['Max_Amount_Donated_Year']=Max_Amount_Donated['amount_gifts']

In [375]:
train['Min_Amount_Donated_Year']=Min_Amount_Donated['amount_gifts']

In [376]:
train['Median_Amount_Donated_Year']=Median_Amount_Donated['amount_gifts']

In [377]:
train['diff_Years_from_first_donation']=Years_from_first_donation['Years_from_first_donation']

In [378]:
train['diff_Years_from_last_donation']=Years_from_last_donation['Years_from_last_donation']

In [379]:
train['Total_freq_Donated']=camp_count['campID']

In [380]:
train['Last_Amount_Donated'] = Last_Amount_Donated['amount_gifts']

In [381]:
train['First_Amount_Donated'] = First_Amount_Donated['amount_gifts']

In [382]:
train['is_donor_M']=donors['is_donor_M']
train['is_donor_F']=donors['is_donor_F']
train['is_donor_C']=donors['is_donor_C']
train['is_donor_S']=donors['is_donor_S']
train['is_donor_U']=donors['is_donor_U']
train['is_donor_French']=donors['is_donor_French']



In [383]:
train['donor_zipcode'] = donors['zipcode']

In [384]:
train['donor_region'] =donors['region']

In [385]:
train['donor_province'] =donors['province']

In [386]:
train['Is_Brussel_province'] = donors['Is_Brussel']
train['Is_Walloon_province'] = donors['Is_Walloon']
train['Is_FlemishBrabant_province'] = donors['Is_FlemishBrabant']
train['Is_Antwerp_province'] = donors['Is_Antwerp']
train['Is_FlemishBrabant2_province'] =donors['Is_FlemishBrabant2']
train['Is_Limburg_province'] = donors['Is_Limburg']
train['Is_Liège_province'] = donors['Is_Liège']
train['Is_Namur_province'] = donors['Is_Namur']
train['Is_Hainaut_province'] = donors['Is_Hainaut']
train['Is_Luxembourg_province'] = donors['Is_Luxembourg']
train['Is_Hainaut2_province'] = donors['Is_Hainaut2']
train['Is_WestFlanders_province'] = donors['Is_WestFlanders'] 
train['Is_EastFlanders_province'] = donors['Is_EastFlanders']

In [387]:
train['Is_Brussel_region'] = donors['Is_Brussel_region']
train['Is_Wallonia_region'] = donors['Is_Wallonia_region']

In [388]:
#Handling NA values

train["total_amount_gifts"] = where(train["total_amount_gifts"].isnull(), 0, train["total_amount_gifts"])
train["max_amount_donated"] = where(train["max_amount_donated"].isnull(), 0, train["max_amount_donated"])
train["min_amount_donated"] = where(train["min_amount_donated"].isnull(), 0, train["min_amount_donated"])
#train["median_amount_donated"] = where(train["median_amount_donated"].isnull(), 0, train["median_amount_donated"])
train["Max_Amount_Donated_Year"] = where(train["Max_Amount_Donated_Year"].isnull(), 0, train["Max_Amount_Donated_Year"])
train["Min_Amount_Donated_Year"] = where(train["Min_Amount_Donated_Year"].isnull(), 0, train["Min_Amount_Donated_Year"])
#train["Median_Amount_Donated_Year"] = where(train["Median_Amount_Donated_Year"].isnull(), 0, train["Median_Amount_Donated_Year"])
train["Last_Amount_Donated"] = where(train["Last_Amount_Donated"].isnull(), 0, train["Last_Amount_Donated"])
train["First_Amount_Donated"] = where(train["First_Amount_Donated"].isnull(), 0, train["First_Amount_Donated"])
train["diff_Years_from_first_donation"] = where(train["diff_Years_from_first_donation"].isnull(), 0, train["diff_Years_from_first_donation"])
train["diff_Years_from_last_donation"] = where(train["diff_Years_from_last_donation"].isnull(), 0, train["diff_Years_from_last_donation"])
train["Total_freq_Donated"] = where(train["Total_freq_Donated"].isnull(), 0, train["Total_freq_Donated"])
train["is_freq_3_camp_attended"] = where(train["is_freq_3_camp_attended"].isnull(), 0, train["is_freq_3_camp_attended"])
train["is_freq_5_camp_attended"] = where(train["is_freq_5_camp_attended"].isnull(), 0, train["is_freq_5_camp_attended"])
train["Donation_last_3_years"] = where(train["Donation_last_3_years"].isnull(), 0, train["Donation_last_3_years"])
train["Donation_last_5_years"] = where(train["Donation_last_5_years"].isnull(), 0, train["Donation_last_5_years"])


In [389]:
train.head()

,donorID,amount_campaign,is_camp_attended,total_amount_gifts,is_donation_year_GE_35,is_donation_year_GE_60,is_donation_year_GE_100,is_freq_3_camp_attended,is_freq_5_camp_attended,Donation_last_3_years,...,Is_Limburg_province,Is_Liège_province,Is_Namur_province,Is_Hainaut_province,Is_Luxembourg_province,Is_Hainaut2_province,Is_WestFlanders_province,Is_EastFlanders_province,Is_Brussel_region,Is_Wallonia_region
0,100001,0.0,1,178.49,1,1,1,1.0,1.0,20.0,...,0,0,0,0,0,0,0,0,0,0
1,100002,0.0,1,20.00,0,0,0,1.0,1.0,26.0,...,0,0,0,0,0,0,0,0,0,0
2,100004,0.0,1,26.00,0,0,0,1.0,1.0,10.0,...,0,0,0,0,0,0,0,0,0,0
3,100008,0.0,1,716.22,1,1,1,1.0,1.0,5.0,...,0,0,0,0,0,0,0,0,1,0
4,100009,0.0,1,14.96,0,0,0,1.0,1.0,70.0,...,0,0,0,0,0,0,0,0,1,0


In [390]:
############################################### TEST DATA ###############################################

In [391]:
# Test dataset
campaign2014 = read_csv(inputData + "/campaign20140115.csv", sep = ';')

In [392]:
#Grouping the data by DonorID to find the total amount donated
campaign2014 = campaign2014.groupby(['donorID'],as_index=False).agg('sum')

In [393]:
#Rename Amount Column
campaign2014 = campaign2014.rename(columns= {'amount': 'amount_campaign'})

In [394]:
# gifts dataset
gifts = read_csv(inputData + "/gifts.csv", sep = ';')

In [395]:
#Convert to date format
gifts['date'] = pandas.to_datetime(gifts['date'])

In [396]:
#filtering data before the campaign date
gifts_test = gifts[gifts.date <= '2014-01-15']

In [397]:
#renaming the amount column in Gifts
gifts_test = gifts_test.rename(columns= {'amount': 'amount_gifts'})

In [398]:
#Camp count per year
camp_count_test = gifts_test.groupby(['donorID'],as_index=False).count()[['campID']]

In [399]:
#column 8
camp_count_test['is_camp_attended'] = where(camp_count_test["campID"] > 0, 1,0)

In [400]:
#column 9
#Total Donation per donor
donation_donor_test = gifts_test.groupby(['donorID'],as_index=False).sum()[['amount_gifts']]

In [401]:
#create a new column for year to obtain the donations per year
gifts_test['gifts_year'] = gifts_test['date'].dt.year

In [402]:
# - to check if it is required qs it has more than one row for donor id
#sum the donations to obtain per year
donation_year_test = gifts_test.groupby(['donorID','gifts_year'],as_index=False).sum()[['amount_gifts']]

In [403]:
donation_year_freq_test = gifts_test.groupby(['donorID'],as_index=False).sum()[['amount_gifts']]

In [404]:
#filtering data for the 3 year frequency rate
freq_3_test = gifts_test[(gifts_test['date'] >= '2011-01-15') & (gifts_test['date'] <= '2014-01-15')]
#column 11
#Camp count freq per 3 years
freq_3_camp_test = freq_3_test.groupby(['donorID'],as_index=False).count()[['campID']]
freq_3_camp_test['is_freq_3_camp_attended'] = where(freq_3_camp_test["campID"] >0, 1,0)
#column 12
#Donation freq per 3 years
Donation_last_3_years_test = freq_3_test.groupby(['donorID'],as_index=False).sum()[['amount_gifts']]

In [405]:
#filtering data for the 5 year frequency rate
freq_5_test = gifts_test[(gifts['date'] >= '2009-01-15') & (gifts_test['date'] <= '2014-01-15')]
#column 13
##Camp count freq per 5 years
freq_5_camp_test = freq_5_test.groupby(['donorID'],as_index=False).count()[['campID']]
freq_5_camp_test['is_freq_5_camp_attended'] = where(freq_5_camp_test["campID"] >0, 1,0)
#column 14
#Donation freq per 5 years
Donation_last_5_years_test = freq_5_test.groupby(['donorID'],as_index=False).sum()[['amount_gifts']]

C:\ProgramData\Anaconda3\lib\site-packages\ipykernel_launcher.py:2: UserWarning: Boolean Series key will be reindexed to match DataFrame index.
  


In [406]:
#column 15
#max donation by a donor
max_donation_test = gifts_test.groupby(['donorID','gifts_year'],as_index=False).sum()
max_donation_test = max_donation_test.groupby(['donorID'],as_index=False).max()[['donorID','amount_gifts']]

In [407]:
#column 16
#min donation by a donor
min_donation_test = gifts_test.groupby(['donorID','gifts_year'],as_index=False).sum()
min_donation_test = min_donation_test.groupby(['donorID'],as_index=False).min()[['donorID','amount_gifts']]

In [408]:
#Median 

In [409]:
#column 18
#Max Amount Paid by a donor
Max_Amount_Donated_test=gifts_test.groupby(['donorID'],as_index=False).max()[['donorID','amount_gifts']]
#column 19
#Min Amount Paid by a donor
Min_Amount_Donated_test=gifts_test.groupby(['donorID'],as_index=False).min()[['donorID','amount_gifts']]
#column 20
#Median Amount Paid by a donor
Median_Amount_Donated_test=gifts_test.groupby(['donorID'],as_index=False).median()[['donorID','amount_gifts']]

In [410]:
#column 21
#Fist Amount Donated
First_Amount_Donated_test=gifts_test.groupby(['donorID','date'],as_index=False).min()[['donorID','amount_gifts','date']]
#column 22
#Last Amount Donated
Last_Amount_Donated_test=gifts_test.groupby(['donorID','date'],as_index=False).max()[['donorID','amount_gifts','date']]

In [411]:
#column 23
#Time since first donation
gb_test = gifts_test.groupby(['donorID'],as_index=False)
gb_test=gb_test.agg({'date' : numpy.min})
gb_update_test= gb_test['date'].dt.year
Years_from_first_donation_test =gb_update_test.to_frame()
Years_from_first_donation_test['Years_from_first_donation'] = 2014-Years_from_first_donation['date']

In [412]:
#column 24
#Time since last donation
gb_test = gifts_test.groupby(['donorID'],as_index=False)
gb_test=gb_test.agg({'date' : numpy.max})
gb_update_test= gb_test['date'].dt.year
Years_from_last_donation_test =gb_update_test.to_frame()
Years_from_last_donation_test['Years_from_last_donation'] = 2014-Years_from_last_donation['date']

In [413]:
test = campaign2014

In [414]:
test['is_camp_attended'] = camp_count_test['is_camp_attended']

In [415]:
test['total_amount_gifts'] = donation_donor_test['amount_gifts']

In [416]:
test['is_donation_year_GE_35'] = where(test['total_amount_gifts'] > 35, 1,0)

In [417]:
test['is_donation_year_GE_60'] = where(test['total_amount_gifts'] > 60, 1,0)

In [418]:
test['is_donation_year_GE_100'] = where(test['total_amount_gifts'] > 100, 1,0)

In [419]:
test['is_freq_3_camp_attended']=freq_3_camp_test['is_freq_3_camp_attended']

In [420]:
test['is_freq_5_camp_attended']=freq_5_camp_test['is_freq_5_camp_attended'] 

In [421]:
test['Donation_last_3_years']=Donation_last_3_years_test['amount_gifts']

In [422]:
test['Donation_last_5_years']=Donation_last_5_years_test['amount_gifts']

In [423]:
test['max_amount_donated']=max_donation_test['amount_gifts']

In [424]:
test['min_amount_donated']=min_donation_test['amount_gifts']

In [425]:
test['Max_Amount_Donated_Year']=Max_Amount_Donated_test['amount_gifts']

In [426]:
test['Min_Amount_Donated_Year']=Min_Amount_Donated_test['amount_gifts']

In [427]:
test['Median_Amount_Donated_Year']=Median_Amount_Donated_test['amount_gifts']

In [428]:
test['diff_Years_from_first_donation']=Years_from_first_donation_test['Years_from_first_donation']

In [429]:
test['diff_Years_from_last_donation']=Years_from_last_donation_test['Years_from_last_donation']

In [430]:
test['Total_freq_Donated']=camp_count_test['campID']

In [431]:
test['Last_Amount_Donated'] = Last_Amount_Donated_test['amount_gifts']

In [432]:
test['First_Amount_Donated'] = First_Amount_Donated_test['amount_gifts']

In [433]:
test['is_donor_M']=donors['is_donor_M']
test['is_donor_F']=donors['is_donor_F']
test['is_donor_C']=donors['is_donor_C']
test['is_donor_S']=donors['is_donor_S']
test['is_donor_U']=donors['is_donor_U']
test['is_donor_French']=donors['is_donor_French']

In [434]:
test['donor_zipcode'] = donors['zipcode']

In [435]:
test['donor_region'] =donors['region']

In [436]:
test['donor_province'] =donors['province']

In [437]:
test['Is_Brussel_province'] = donors['Is_Brussel']
test['Is_Walloon_province'] = donors['Is_Walloon']
test['Is_FlemishBrabant_province'] = donors['Is_FlemishBrabant']
test['Is_Antwerp_province'] = donors['Is_Antwerp']
test['Is_FlemishBrabant2_province'] =donors['Is_FlemishBrabant2']
test['Is_Limburg_province'] = donors['Is_Limburg']
test['Is_Liège_province'] = donors['Is_Liège']
test['Is_Namur_province'] = donors['Is_Namur']
test['Is_Hainaut_province'] = donors['Is_Hainaut']
test['Is_Luxembourg_province'] = donors['Is_Luxembourg']
test['Is_Hainaut2_province'] = donors['Is_Hainaut2']
test['Is_WestFlanders_province'] = donors['Is_WestFlanders'] 
test['Is_EastFlanders_province'] = donors['Is_EastFlanders']
test['Is_Brussel_region'] = donors['Is_Brussel_region']
test['Is_Wallonia_region'] = donors['Is_Wallonia_region']

In [438]:
test['Is_Brussel_region'] = donors['Is_Brussel_region']
test['Is_Wallonia_region'] = donors['Is_Wallonia_region']

In [440]:
#Handling NA values

test["total_amount_gifts"] = where(test["total_amount_gifts"].isnull(), 0, test["total_amount_gifts"])
test["max_amount_donated"] = where(test["max_amount_donated"].isnull(), 0, test["max_amount_donated"])
test["min_amount_donated"] = where(test["min_amount_donated"].isnull(), 0, test["min_amount_donated"])
#test["median_amount_donated"] = where(test["median_amount_donated"].isnull(), 0, test["median_amount_donated"])
test["Max_Amount_Donated_Year"] = where(test["Max_Amount_Donated_Year"].isnull(), 0, test["Max_Amount_Donated_Year"])
test["Min_Amount_Donated_Year"] = where(test["Min_Amount_Donated_Year"].isnull(), 0, test["Min_Amount_Donated_Year"])
#test["Median_Amount_Donated_Year"] = where(test["Median_Amount_Donated_Year"].isnull(), 0, test["Median_Amount_Donated_Year"])
test["Last_Amount_Donated"] = where(test["Last_Amount_Donated"].isnull(), 0, test["Last_Amount_Donated"])
test["First_Amount_Donated"] = where(test["First_Amount_Donated"].isnull(), 0, test["First_Amount_Donated"])
test["diff_Years_from_first_donation"] = where(test["diff_Years_from_first_donation"].isnull(), 0, test["diff_Years_from_first_donation"])
test["diff_Years_from_last_donation"] = where(test["diff_Years_from_last_donation"].isnull(), 0, test["diff_Years_from_last_donation"])
test["Total_freq_Donated"] = where(test["Total_freq_Donated"].isnull(), 0, test["Total_freq_Donated"])
test["is_freq_3_camp_attended"] = where(test["is_freq_3_camp_attended"].isnull(), 0, test["is_freq_3_camp_attended"])
test["is_freq_5_camp_attended"] = where(test["is_freq_5_camp_attended"].isnull(), 0, test["is_freq_5_camp_attended"])
test["Donation_last_3_years"] = where(test["Donation_last_3_years"].isnull(), 0, test["Donation_last_3_years"])
test["Donation_last_5_years"] = where(test["Donation_last_5_years"].isnull(), 0, test["Donation_last_5_years"])

In [441]:
test.head(10)

,donorID,amount_campaign,is_camp_attended,total_amount_gifts,is_donation_year_GE_35,is_donation_year_GE_60,is_donation_year_GE_100,is_freq_3_camp_attended,is_freq_5_camp_attended,Donation_last_3_years,...,Is_Limburg_province,Is_Liège_province,Is_Namur_province,Is_Hainaut_province,Is_Luxembourg_province,Is_Hainaut2_province,Is_WestFlanders_province,Is_EastFlanders_province,Is_Brussel_region,Is_Wallonia_region
0,100001,0.0,1,178.49,1,1,1,1.0,1.0,42.0,...,0,0,0,0,0,0,0,0,0,0
1,100002,0.0,1,20.00,0,0,0,1.0,1.0,20.0,...,0,0,0,0,0,0,0,0,0,0
2,100008,0.0,1,42.00,1,0,0,1.0,1.0,5.0,...,0,0,0,0,0,0,0,0,0,0
3,100009,0.0,1,716.22,1,1,1,1.0,1.0,40.0,...,0,0,0,0,0,0,0,0,1,0
4,100010,0.0,1,24.96,0,0,0,1.0,1.0,40.0,...,0,0,0,0,0,0,0,0,1,0
5,100011,0.0,1,5.00,0,0,0,1.0,1.0,20.0,...,0,0,0,0,0,0,0,0,1,0
6,100012,0.0,1,429.16,1,1,1,1.0,1.0,5.0,...,0,0,0,0,0,0,0,0,1,0
7,100018,0.0,1,124.37,1,1,1,1.0,1.0,6.0,...,0,0,0,0,0,0,0,0,0,0
8,100019,0.0,1,326.78,1,1,1,1.0,1.0,10.0,...,0,0,0,0,0,0,0,0,1,0
9,100021,0.0,1,40.00,1,0,0,1.0,1.0,86.0,...,0,0,0,0,0,0,0,0,1,0


In [442]:
test.to_csv(r'C:\Users\dkalapati\Desktop\predictive and descriptive\casestudy\data\test.csv')